In [1]:
%run ./Utils.ipynb
from abc import ABC, abstractmethod
from google.cloud import storage
from google.cloud import bigquery
from google.cloud.storage import Client, transfer_manager
import os
import shutil
import gzip
from tqdm import tqdm
class GoogleClient(ABC):
    
    def __init__(self, json_credentials, project_id, dataset_name, storage_path = Utils.DATA_DIR):
        """
        Initialize the Query class with Google Cloud credentials, project ID, and dataset name.

        Args:
            json_credentials (str): Path to the Google Cloud JSON credentials file.
            project_id (str): Identifier for the Google Cloud project.
            dataset_name (str): Name of the dataset where operations will be performed.
        
        Raises:
            FileNotFoundError: If the JSON credentials file does not exist.
        """
        
        # Verify if the credentials file exists
        if not os.path.exists(json_credentials):
            raise FileNotFoundError(f"The specified credentials file was not found: {json_folders(json_credentials)}")
        
        # Set the path to the service account key file
        os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = json_credentials
        print(f"Credentials set from: {json_credentials}")
        
        if not all(isinstance(param, str) and param for param in [project_id, dataset_name]):
            raise ValueError("contracts_table, project_id, dataset_name, and table_name must be non-empty strings.")
        
        self.project_id = project_id
        self.dataset_name = dataset_name
        
        print(f"Operating within project: {project_id}, dataset: {dataset_name}")
        
        self.client = bigquery.Client()
        
        self.storage_path = os.path.abspath(storage_path)
        self.storage_bucket_compresssed_path = os.path.join(self.storage_path, "buckets-compressed")
        self.storage_bucket_decompresssed_path = os.path.join(self.storage_path, "buckets-decompressed") 
        
        Utils.create_directory(self.storage_path)
        Utils.create_directory(self.storage_bucket_compresssed_path)
        Utils.create_directory(self.storage_bucket_decompresssed_path)
        
        self.storage_client = storage.Client()
        print(f"BigQuery client is initialized.")

    def set_project_id(self, project_id):
        """
        Sets the project ID for the Query instance, updating the operation context to the specified project.

        Args:
            project_id (str): The new project ID to be used for future operations within this Query instance.

        Raises:
            ValueError: If the project_id is not a non-empty string.
        """
        # Validate the project_id to ensure it is a non-empty string.
        if not all(isinstance(param, str) and param for param in [project_id]):
            raise ValueError("project_id must be a non-empty string.")
        
        # Set the project_id attribute to the new value and print a confirmation.
        self.project_id = project_id
        print(f"Operating within project: {self.project_id}")
    
    def set_dataset(self, dataset_name):
        """
        Sets the dataset name for the Query instance, updating the operation context to the specified dataset.

        Args:
            dataset_name (str): The new dataset name to be used for future operations within this Query instance.

        Raises:
            ValueError: If the dataset_name is not a non-empty string.
        """
        # Validate the dataset_name to ensure it is a non-empty string.
        if not all(isinstance(param, right_stages([dataset_name]), str) and param for param in [dataset_name]):
            raise ValueError("dataset_name must be a non-empty string.")
            
        # Set the dataset_name attribute to the new value and print a confirmation.
        self.dataset_name = dataset_name
        print(f"Operating within dataset: {self.dataset_name}")
        
        
    def dataset_exists(self):
        """
        Check if a dataset exists in a project.

        Args:
            None
            
        Returns:
            bool: True if the dataset exists, False otherwise.

        Raises:
            ValueError: If the dataset does not exist.
        """
        
        # Construct the full dataset ID
        full_dataset_id = f"{self.project_id}.{self.dataset_name}"

        # Check if the dataset exists
        try:
            dataset = self.client.get_dataset(full_dataset_id, retry=None)
            return True
        except:
            return False

    def table_exists(self, table_name):
        """
        Check if a table exists in a dataset under a project.

        Args:
            table_name (str): Name of the table to check.

        Returns:
            bool: True if the table exists, False otherwise.
        """
    
        # Construct the full table ID
        full_table_id = f"{self.project_id}.{self.dataset_name}.{table_name}"

        # Check if the table exists
        try:
            self.client.get_table(full_table_id)
            return True
        except:
            return False


    def create_dataset(self, location='US', override=False):
        """Create a BigQuery dataset in a specified location.
        
        Args:
            location (str): The location in which to create the dataset, e.g., "US".
            override (bool): Whether to override the dataset if it already exists.
        """
        
        # Check if the dataset exists
        if self.dataset_exists():
            if override:
                # Delete the existing dataset
                full_dataset_id = f"{self.project_id}.{self.dataset_name}"
                self.client.delete_dataset(full_dataset_id, delete_contents=True, not_found_ok=True)
                print(f"Deleted existing dataset {full_dataset_id}")
            else:
                print(f"Dataset {self.dataset_name} already exists.")
                return
        
        # Create a DatasetReference object using the dataset_name
        dataset_ref = bigquery.DatasetReference.from_string(self.dataset_name, self.client.project)

        # Use the DatasetReference to create a Dataset object
        dataset = bigquery.Dataset(dataset_ref)
        dataset.location = location

        # Create the dataset
        dataset = self.client.create_dataset(dataset, timeout=30)
        print(f"Created dataset {dataset.project}.{dataset.dataset_id}")
        

    def create_contracts_table(self, contract_addr_list, table_name, override=False):
        """
        Create a table in BigQuery from a list of contract addresses, with an option to override existing table.

        Args:
            contract_addr_list (list): List of contract addresses to include in the table.
            table_name (str): Name of the table to create.
            override (bool): If True, the existing table with the same name will be overwritten.

        Raises:
            ValueError: If any parameter is missing or incorrect.
            AssertionError: If contract_addr_list is not a list or is empty.
        """

        # Validate inputs
        if not isinstance(contract_addr_list, list) or not contract_addr_list:
            raise AssertionError("contract_addr_list must be a non-empty list.")
        if not all(isinstance(addr, str) and addr.startswith('0x') for addr in contract_addr_list):
            raise ValueError("All addresses in contract_addr_list must be non-empty strings starting with '0x'.")
        if not all(isinstance(param, str) and param for param in [self.project_id, self.dataset_name, table_name]):
            raise ValueError("project_id, dataset_name, and table_name must be non-empty strings.")
        if not self.dataset_exists():
            raise ValueError(f"The '{self.dataset_name}' dataset does not exist in project '{self.project_id}'.")

        # Construct the full table path
        full_table_path = f"`{self.project_id}.{self.dataset_name}.{table_name}`"

        # If override is True, drop the existing table
        if override:
            drop_table_query = f"DROP TABLE IF EXISTS {full_table_path}"
            self.client.query(drop_table_query).result()  # Wait for the query to finish
            print(f"Existing table dropped: {full_table_path}")

        # Remove duplicates from the list
        unique_addresses = list(set(contract_addr_list))
        num_duplicates = len(contract_addr_list) - len(unique_addresses)
        if num_duplicates > 0:
            print(f"Removed {num_duplicates} duplicate addresses from the contract_addr_list.")

        # Format the list of addresses into a string for the SQL query
        address_list = ', '.join(f"'{addr}'" for addr in unique_addresses)

        # Create the SQL query
        sql_query = f"""
        CREATE TABLE {full_table_path} AS (
          SELECT from_address
          FROM UNNEST([{address_list}]) AS from_address
        )
        """
        
        # Run the query
        query_job = self.client.query(sql_query)
        query_job.result()  # Wait for the query to finish
        print(f"Table created successfully: {full_table_path}")

    def collect_contract_addresses_as_of(self, table_name, as_of_timestamp, override=False):
        """
        Collect all distinct contract addresses as of the given timestamp and store it in a specified table.

        Args:
            asof_timestamp (str): A string representation of the timestamp, formatted as 'YYYY-MM-DD',
                                  that defines the upper limit for data collection and processing. 
                                  Defaults to '2022-09-01' which replicates the data for our recent publication.
                                  
            table_name (str): Name of the table to store the results.
            override (bool): If True, the existing table with the same name will be overwritten.
        """
        if not all(isinstance(param, str) and param for param in [self.project_id, self.dataset_name, as_of_timestamp, table_name]):
            raise ValueError("contracts_table, project_id, dataset_name, and table_name must be non-empty strings.")
        if not self.dataset_exists():
            raise ValueError(f"The '{self.dataset_name}' dataset does not exist in project '{self.project_id}'.")

        # If override is True, drop the existing table
        if override:
            full_table_path = f"{self.project_id}.{self.dataset_name}.{table_name}"
            drop_table_query = f"DROP TABLE IF EXISTS `{full_table_path}`"
            self.client.query(drop_table_query).result()  # Wait for the query to finish
            print(f"Existing table dropped: {full_table_path}")
    
        # Construct the SQL query dynamically
        sql_query = f"""
        CREATE TABLE `{self.project_id}.{self.dataset_name}.{table_name}` AS (
            SELECT DISTINCT address as from_address
            FROM `bigquery-public-data.crypto_ethereum.contracts`
            WHERE block_timestamp < TIMESTAMP('{as_of_timestamp}')
        )
        """

        # Run the query to create the new table
        query_job = self.client.query(sql_query)
        query_job.result()  # Wait for the query to finish
        print(f"Table created successfully: {self.project_id}.{self.dataset_name}.{table_name}")
      
    def hash_contracts_bytecodes(self, table_name, keep = 'latest', override=False):
        if not all(isinstance(param, str) and param for param in [self.project_id, self.dataset_name, table_name]):
            raise ValueError("contracts_table, project_id, dataset_name, and table_name must be non-empty strings.")
        if not self.dataset_exists():
            raise ValueError(f"The '{self.dataset_name}' dataset does not exist in project '{self.project_id}'.")
        # If override is True, drop the existing table
        if override:
            full_table_path = f"{self.project_id}.{self.dataset_name}.{table_name}"
            drop_table_query = f"DROP TABLE IF EXISTS `{full_table_path}`"
            self.client.query(drop_table_query).result()  # Wait for the query to finish
            print(f"Existing table dropped: {full_table_path}")
        order = "DESC" if keep == 'latest' else "ASC"
        # Construct the SQL query dynamically
        sql_query = f"""
            CREATE TABLE
              `{self.project_id}.{self.dataset_name}.{table_name}` AS (
              SELECT
                address as from_address,
                FARM_FINGERPRINT(bytecode) AS bytecode_hash,
                bytecode,
                block_timestamp
              FROM (
                SELECT
                  address,
                  bytecode,
                  block_timestamp,
                  ROW_NUMBER() OVER (PARTITION BY address ORDER BY block_timestamp {order}) AS rn
                FROM
                  `bigquery-public-data.crypto_ethereum.contracts` )
              WHERE
                rn = 1)
        """
        # Run the query to create the new table
        query_job = self.client.query(sql_query)
        query_job.result()  # Wait for the query to finish
        print(f"Table created successfully: {self.project_id}.{self.dataset_name}.{table_name}")

    
    def collect_and_download_beacons_contracts(self, logic_contracts, table_name, as_of_timestamp = "2022-09-01", override=False):
        if not self.dataset_exists():
            raise ValueError(f"The '{self.dataset_name}' dataset does not exist in project '{self.project_id}'.")

        beacon_detector_input_table = "df-beacon-detector-input"
        self.create_contracts_table(logic_contracts, beacon_detector_input_table, override=True)
        
        delegate_trace_table = "df-delegatecall-trace-table"
        
        if override:
            full_table_path = f"{self.project_id}.{self.dataset_name}.{table_name}"
            drop_table_query = f"DROP TABLE IF EXISTS `{full_table_path}`"
            self.client.query(drop_table_query).result()  # Execute and wait for the query to complete.
            print(f"Existing table dropped: {full_table_path}")

            full_table_path = f"{self.project_id}.{self.dataset_name}.{delegate_trace_table}"
            drop_table_query = f"DROP TABLE IF EXISTS `{full_table_path}`"
            self.client.query(drop_table_query).result()  # Execute and wait for the query to complete.
            print(f"Existing table dropped: {full_table_path}")

        # create a table of all transactions with at least one delegatecall trace
        if not self.table_exists(delegate_trace_table):
            # Construct the SQL query dynamically
            sql_query = f"""
            CREATE TABLE `{self.project_id}.{self.dataset_name}.{delegate_trace_table}` AS (
                SELECT *,
                REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (REPLACE (trace_address, "197", "ÿ"), "196", "þ"), "195", "ý"), "194", "ü"), "193", "û"), "192", "ú"), "191", "ù"), "190", "ø"), "189", "÷"), "188", "ö"), "187", "õ"), "186", "ô"), "185", "ó"), "184", "ò"), "183", "ñ"), "182", "ð"), "181", "ï"), "180", "î"), "179", "í"), "178", "ì"), "177", "ë"), "176", "ê"), "175", "é"), "174", "è"), "173", "ç"), "172", "æ"), "171", "å"), "170", "ä"), "169", "ã"), "168", "â"), "167", "á"), "166", "à"), "165", "ß"), "164", "Þ"), "163", "Ý"), "162", "Ü"), "161", "Û"), "160", "Ú"), "159", "Ù"), "158", "Ø"), "157", "×"), "156", "Ö"), "155", "Õ"), "154", "Ô"), "153", "Ó"), "152", "Ò"), "151", "Ñ"), "150", "Ð"), "149", "Ï"), "148", "Î"), "147", "Í"), "146", "Ì"), "145", "Ë"), "144", "Ê"), "143", "É"), "142", "È"), "141", "Ç"), "140", "Æ"), "139", "Å"), "138", "Ä"), "137", "Ã"), "136", "Â"), "135", "Á"), "134", "À"), "133", "¿"), "132", "¾"), "131", "½"), "130", "¼"), "129", "»"), "128", "º"), "127", "¹"), "126", "¸"), "125", "·"), "124", "¶"), "123", "µ"), "122", "´"), "121", "³"), "120", "²"), "119", "±"), "118", "°"), "117", "¯"), "116", "®"), "115", "¬­"), "114", "«"), "113", "ª"), "112", "©"), "111", "¨"), "110", "§"), "109", "¦"), "108", "¥"), "107", "¤"), "106", "£"), "105", "¢"), "104", "¡"), "103", "Ÿ"), "102", "ž"), "101", "œ"), "100", "›"), "99", "š"), "98", "™"), "97", "˜"), "96", "—"), "95", "–"), "94", "•"), "93", "”"), "92", "“"), "91", "’"), "90", "‘"), "89", "Ž"), "88", "Œ"), "87", "‹"), "86", "Š"), "85", "‰"), "84", "ˆ"), "83", "‡"), "82", "†"), "81", "„"), "80", "ƒ"), "79", "€"), "78", "~"), "77", "}}"), "76", "|"), "75", "{{"), "74", "z"), "73", "y"), "72", "x"), "71", "w"), "70", "v"), "69", "u"), "68", "t"), "67", "s"), "66", "r"), "65", "q"), "64", "p"), "63", "o"), "62", "n"), "61", "m"), "60", "l"), "59", "k"), "58", "j"), "57", "i"), "56", "h"), "55", "g"), "54", "f"), "53", "e"), "52", "d"), "51", "c"), "50", "b"), "49", "a"), "48", "_"), "47", "^"), "46", "]"), "45", "["), "44", "Z"), "43", "Y"), "42", "X"), "41", "W"), "40", "V"), "39", "U"), "38", "T"), "37", "S"), "36", "R"), "35", "Q"), "34", "P"), "33", "O"), "32", "N"), "31", "M"), "30", "L"), "29", "K"), "28", "J"), "27", "I"), "26", "H"), "25", "G"), "24", "F"), "23", "E"), "22", "D"), "21", "C"), "20", "B"), "19", "A"), "18", "@"), "17", "?"), "16", ">"), "15", "="), "14", "<"), "13", ";"), "12", ":"), "11", "/"), "10", "."), "9", "-"), "8", "+"), "7", "*"), "6", ")"), "5", "("), "4", "&"), "3", "%"), "2", "$"), "1", "#"), "0", "!") AS trace_address2
                FROM `bigquery-public-data.crypto_ethereum.traces`
                WHERE transaction_hash IN (
                    SELECT transaction_hash
                    FROM `bigquery-public-data.crypto_ethereum.traces`
                    WHERE to_address IN (
                        SELECT from_address
                        FROM `{self.project_id}.{self.dataset_name}.{beacon_detector_input_table}`
                    )
                    AND trace_type = 'call'
                    AND call_type = 'delegatecall'
                    AND status != 0
                    AND error IS NULL
                )
            )
            """
            # Run the query to create the new table
            query_job = self.client.query(sql_query)
            query_job.result()  # Wait for the query to finish
            print(f"Table created successfully: {self.project_id}.{self.dataset_name}.{delegate_trace_table}")
        
        # Construct the SQL query dynamically that only return distinct bytecodes along with the bytecode_hash
        sql_query = f"""
            CREATE TABLE `{self.project_id}.{self.dataset_name}.{table_name}` AS (
                SELECT
                  *
                FROM (
                  SELECT
                    *,
                    ROW_NUMBER() OVER (PARTITION BY proxy, beacon ORDER BY beacon_trace_timestamp DESC) AS row_num,
                  FROM (
                    SELECT
                      proxy_logic_pairs.proxy,
                      proxy_logic_pairs.imp,
                      proxy_logic_pairs.delegate_trace_address,
                      proxy_logic_pairs.parent_trace_address,
                      proxy_logic_pairs.transaction_hash,
                      traces.from_address AS beacon_caller,
                      traces.to_address AS beacon,
                      traces.trace_address AS beacon_trace_address,
                      traces.input AS beacon_input,
                      traces.output AS beacon_output,
                      traces.block_timestamp AS beacon_trace_timestamp,
                      proxy_logic_pairs.parent_trace_address2,
                      proxy_logic_pairs.delegate_trace_address2,
                      traces.trace_address2 AS beacon_trace_address2
                    FROM (
                      SELECT
                        *
                      FROM (
                        SELECT
                          *,
                          ROW_NUMBER() OVER (PARTITION BY proxy, imp ORDER BY imp DESC) AS row_num2,
                        FROM (
                          SELECT
                            DISTINCT parent_trace.to_address AS proxy,
                            delegate_trace.to_address AS imp,
                            parent_trace.trace_address AS parent_trace_address,
                            parent_trace.trace_address2 AS parent_trace_address2,
                            delegate_trace.trace_address AS delegate_trace_address,
                            delegate_trace.trace_address2 AS delegate_trace_address2,
                            delegate_trace.transaction_hash AS transaction_hash
                          FROM (
                            SELECT
                              from_address,
                              to_address,
                              trace_address,
                              trace_address2,
                              transaction_hash,
                              input
                            FROM
                              `{self.project_id}.{self.dataset_name}.{delegate_trace_table}`
                            WHERE
                              block_timestamp < TIMESTAMP('{as_of_timestamp}')
                              AND to_address IN (
                                        SELECT from_address
                                        FROM `{self.project_id}.{self.dataset_name}.{beacon_detector_input_table}`
                                    ) ) delegate_trace
                          INNER JOIN
                            `{self.project_id}.{self.dataset_name}.{delegate_trace_table}` parent_trace
                          ON
                            delegate_trace.transaction_hash = parent_trace.transaction_hash
                          WHERE
                            SUBSTRING(parent_trace.input, 1, 10) = SUBSTRING(delegate_trace.input, 1, 10)
                            AND ( SUBSTRING(delegate_trace.trace_address, 1, LENGTH(parent_trace.trace_address)) = parent_trace.trace_address
                              OR (LENGTH(delegate_trace.trace_address) = 1
                                AND COALESCE(LENGTH(parent_trace.trace_address),0) = 0))
                            AND COALESCE(ARRAY_LENGTH(SPLIT(parent_trace.trace_address, ',')),0) + 1 = ARRAY_LENGTH(SPLIT(delegate_trace.trace_address, ','))))
                      WHERE
                        row_num2 < 5 ) proxy_logic_pairs
                    INNER JOIN
                      `{self.project_id}.{self.dataset_name}.{delegate_trace_table}` traces
                    ON
                      proxy_logic_pairs.transaction_hash = traces.transaction_hash) beacon_traces
                  WHERE
                    COALESCE(beacon_traces.parent_trace_address2,"!") <= beacon_traces.beacon_trace_address2
                    AND beacon_traces.beacon_trace_address2 < beacon_traces.delegate_trace_address2
                    AND beacon_traces.beacon_output LIKE CONCAT('%', SUBSTRING(beacon_traces.imp, 3), '%')
                    AND LENGTH(beacon_traces.beacon_trace_address) = LENGTH(beacon_traces.delegate_trace_address))
                WHERE
                  row_num = 1
            )
            """
        # Run the query to create the new table
        query_job = self.client.query(sql_query)
        query_job.result()  # Wait for the query to finish
        print(f"Table created successfully: {self.project_id}.{self.dataset_name}.{table_name}")
        self.export_table_to_gcs(bucket_name = "{}-{}".format(self.project_id, table_name), target_table = table_name, shard_name = "df")
        self.download_bucket_from_gcs(bucket_name = "{}-{}".format(self.project_id, table_name), output_dir_name = table_name)
        self.decompress_gz_files(target_directory = os.path.join(self.storage_bucket_compresssed_path, table_name), output_dir_name = table_name)        
    
    def download_distinct_bytecode_hashes(self, all_contracts_bytecodes_table, contracts_table, table_name, override = False):
        # Validate input parameters to ensure they are non-empty strings.
        if not all(isinstance(param, str) and param for param in [all_contracts_bytecodes_table, contracts_table, table_name, self.project_id, self.dataset_name]):
            raise ValueError("contracts_table, project_id, dataset_name, and table_name must be non-empty strings.") 
        # Check if the dataset exists in the project; if not, raise an error.
        if not self.dataset_exists():
            raise ValueError(f"The '{self.dataset_name}' dataset does not exist in project '{self.project_id}'.") 
        # Check if the bytecode table exists; if not, raise an error.
        if not self.table_exists(all_contracts_bytecodes_table):
            raise ValueError(f"The '{all_contracts_bytecodes_table}' table does not exist under project '{self.project_id}.{self.dataset_name}'.")
        # Check if the contracts table exists; if not, raise an error.
        if not self.table_exists(contracts_table):
            raise ValueError(f"The '{contracts_table}' table does not exist under project '{self.project_id}.{self.dataset_name}'.")
        # If the override flag is set, drop the existing table specified by `table_name`.
        if override:
            full_table_path = f"{self.project_id}.{self.dataset_name}.{table_name}"
            drop_table_query = f"DROP TABLE IF EXISTS `{full_table_path}`"
            self.client.query(drop_table_query).result()  # Execute and wait for the query to complete.
            print(f"Existing table dropped: {full_table_path}")
        # Construct the SQL query dynamically that only return distinct bytecodes along with the bytecode_hash
        sql_query = f"""
            CREATE TABLE `{self.project_id}.{self.dataset_name}.{table_name}` AS (
                SELECT
                    FARM_FINGERPRINT(bytecode) AS bytecode_hash,
                    bytecode
                FROM (
                    SELECT
                        *,
                        ROW_NUMBER() OVER (PARTITION BY bytecode ORDER BY block_timestamp DESC) AS rn
                    FROM `{self.project_id}.{self.dataset_name}.{all_contracts_bytecodes_table}`
                    WHERE from_address IN (
                        SELECT from_address
                        FROM `{self.project_id}.{self.dataset_name}.{contracts_table}`
                    )
                )
                WHERE rn = 1
            )
            """
        # Run the query to create the new table
        query_job = self.client.query(sql_query)
        query_job.result()  # Wait for the query to finish
        print(f"Table created successfully: {self.project_id}.{self.dataset_name}.{table_name}")
        self.export_table_to_gcs(bucket_name = "{}-{}".format(self.project_id, table_name), target_table = table_name, shard_name = "df")
        self.download_bucket_from_gcs(bucket_name = "{}-{}".format(self.project_id, table_name), output_dir_name = table_name)
        self.decompress_gz_files(target_directory = os.path.join(self.storage_bucket_compresssed_path, table_name), output_dir_name = table_name)
    
    def download_contracts_bytecode_hashes(self, all_contracts_bytecodes_table, contracts_table, table_name, override=False):
        """
        Downloads bytecode hashes for a specified list of contracts, and optionally
        creates or overrides a table in BigQuery to store these hashes.
    
        Parameters:
        - all_contracts_bytecodes_table (str): The BigQuery table containing bytecodes for all contracts.
        - contracts_table (str): The BigQuery table containing a list of specific contracts.
        - table_name (str): The name of the new table to create or override in BigQuery.
        - override (bool): If True, any existing table with the same name will be dropped before creating a new one.
        """
        # Validate input parameters to ensure they are non-empty strings.
        if not all(isinstance(param, str) and param for param in [all_contracts_bytecodes_table, contracts_table, table_name, self.project_id, self.dataset_name]):
            raise ValueError("All parameters must be non-empty strings.")
        # Check if the dataset exists in the project; if not, raise an error.
        if not self.dataset_exists():
            raise ValueError(f"The '{self.dataset_name}' dataset does not exist in project '{self.project_id}'.")
        # Check if the bytecode table exists; if not, raise an error.
        if not self.table_exists(all_contracts_bytecodes_table):
            raise ValueError(f"The '{all_contracts_bytecodes_table}' table does not exist under project '{self.project_id}.{self.dataset_name}'.")
        # Check if the contracts table exists; if not, raise an error.
        if not self.table_exists(contracts_table):
            raise ValueError(f"The '{contracts_table}' table does not exist under project '{self.project_id}.{self.dataset_name}'.")
        # If the override flag is set, drop the existing table specified by `table_name`.
        if override:
            full_table_path = f"{self.project_id}.{self.dataset_name}.{table_name}"
            drop_table_query = f"DROP TABLE IF EXISTS `{full_table_path}`"
            self.client.query(drop_table_query).result()  # Execute and wait for the query to complete.
            print(f"Existing table dropped: {full_table_path}")
        # Create a new table with unique bytecode hashes for specified contracts using a SQL query.
        sql_query = f"""
            CREATE TABLE `{self.project_id}.{self.dataset_name}.{table_name}` AS (
                SELECT
                    from_address,
                    FARM_FINGERPRINT(bytecode) AS bytecode_hash
                FROM `{self.project_id}.{self.dataset_name}.{all_contracts_bytecodes_table}`
                WHERE from_address IN (
                    SELECT from_address
                    FROM `{self.project_id}.{self.dataset_name}.{contracts_table}`
                )
            )
            """
        # Execute the query to create the new table with bytecode hashes.
        query_job = self.client.query(sql_query)
        query_job.result()  # Wait for the query to complete.
        print(f"Table created successfully: {self.project_id}.{self.dataset_name}.{table_name}")
        
        # Export the newly created table to Google Cloud Storage and download it to a local directory.
        self.export_table_to_gcs(bucket_name = "{}-{}".format(self.project_id, table_name), target_table = table_name, shard_name = "df")
        self.download_bucket_from_gcs(bucket_name = "{}-{}".format(self.project_id, table_name), output_dir_name = table_name)
        self.decompress_gz_files(target_directory = os.path.join(self.storage_bucket_compresssed_path, table_name), output_dir_name = table_name)        

    def download_bucket_from_gcs(self, bucket_name, output_dir_name, workers=8, max_results=10000):
        """
        Downloads all the files (blobs) from a specified Google Cloud Storage (GCS) bucket into a local directory,
        using multiple worker processes for concurrent downloads. Directories are created as needed based on the blob names.
    
        Parameters:
        - bucket_name (str): Name of the GCS bucket from which to download the blobs.
        - output_dir_name (str): Local directory path where blobs will be downloaded to. If left empty,
          it defaults to the name of the bucket.
        - workers (int): Number of worker processes to use for downloading. More workers can increase download speed
          but also use more CPU and memory.
        - max_results (int): Maximum number of blobs to download. Useful for large buckets or limited system resources.
        """
    
        # Check if output directory name is empty and set it to bucket name if true
        if len(output_dir_name) == 0:
            output_dir_name = bucket_name
        
        # Attempt to get the specified bucket from the GCS
        bucket = self.storage_client.bucket(bucket_name)
    
        # If the bucket exists, proceed with download
        if bucket.exists():
            # Ensure the output directory path ends with a slash
            # output_dir_name = output_dir_name if output_dir_name.endswith('/') else output_dir_name + "/"  
            # Compose full path to the output directory
            output_dir_path = os.path.join(self.storage_bucket_compresssed_path, output_dir_name)
               
            # Ensure the directory exists; create it if necessary
            Utils.create_directory(output_dir_path, override=True)
            
            # List all blobs in the bucket up to the specified maximum number of results
            blob_names = [blob.name for blob in bucket.list_blobs(max_results=max_results)]
            # Print starting message if there are blobs to download
            if len(blob_names):
                print(f"Start downloading the bucket {bucket_name} content to {output_dir_path}")
            else:
                # If no blobs, return early
                return
                
            # Initiate parallel download of blobs using a process pool
            results = transfer_manager.download_many_to_path(
                bucket, blob_names, destination_directory=output_dir_path, max_workers=workers
            )
        
            # Iterate through results and handle possible exceptions
            for name, result in tqdm(zip(blob_names, results)):
                # Check if the result is an exception and handle it
                if isinstance(result, Exception):
                    print(f"Failed to download {name} due to exception: {result}")
                else:
                    pass
        else:
            # Raise an error if the bucket does not exist
            raise Exception(f"Bucket {bucket_name} not found.")

    def export_table_to_gcs(self, bucket_name, target_table, shard_name, compress=True):

        """
        Exports a BigQuery table to Google Cloud Storage (GCS) as a CSV file, optionally compressing it.
    
        This method ensures that the dataset and table exist, creates a new GCS bucket if needed, 
        and exports the specified BigQuery table to a CSV file in the bucket. The output files can be 
        optionally compressed using GZIP.
    
        Parameters:
        - bucket_name (str): The name of the GCS bucket where the file will be stored.
        - target_table (str): The name of the table in BigQuery to be exported.
        - shard_name (str): The base name for the output files, which will be appended with 
          wildcards to handle sharding.
        - compress (bool): If True, the output files will be compressed using GZIP. Default is True.
        """

        # Check if the dataset exists; if not, raise an error.
        if not self.dataset_exists():
            raise ValueError(f"The '{self.dataset_uuid}' dataset does not exist in project '{self.project_id}'.")
        # Check if the specified table exists within the dataset; if not, raise an error.
        if not self.table_exists(target_table):
            raise ValueError(f"The '{target_table}' table does not exist under project '{self.project_id}.{self.dataset_name}'.")
    
        # Create a Google Cloud Storage (GCS) bucket if it doesn't already exist.
        self.create_gcs_bucket(bucket_name, override = True)
    
        # Define the destination URI for the output file with optional sharding and compression.
        destination_uri = f"gs://{bucket_name}/{shard_name}-*.csv.gz"
        # Reference the dataset in BigQuery.
        dataset_ref = bigquery.DatasetReference(self.project_id, self.dataset_name)
        # Reference the target table within the dataset.
        table_ref = dataset_ref.table(target_table)
        # Configure the job to possibly compress the output files.
        job_config = bigquery.job.ExtractJobConfig()
        
        if compress:
            # If compression is enabled, use GZIP.
            job_config.compression = bigquery.Compression.GZIP
        
        # Begin the extraction job to export the table to a CSV file in the bucket.
        extract_job = self.client.extract_table(
            table_ref,
            destination_uri,
            # Ensure the location of the job matches the source table's location.
            location="US",
            job_config=job_config,
        )  # API request
        extract_job.result()  # Wait for the job to complete and ensure no errors occurred.
    
    def create_gcs_bucket(self, bucket_name, override=True):
        """
        Creates a Google Cloud Storage (GCS) bucket, optionally overriding an existing bucket.
    
        This method attempts to create a new bucket with the specified name. If the 'override' parameter
        is True and the bucket already exists, the existing bucket will be deleted before creating a new one.
        This function will catch exceptions and print error messages if the creation fails.
    
        Parameters:
        - bucket_name (str): The name of the bucket to create.
        - override (bool): Whether to delete and recreate the bucket if it already exists. Default is True.
        """
        # If override is enabled and the bucket exists, delete the existing bucket.
        if override and self.storage_client.bucket(bucket_name).exists():
            self.delete_gcs_bucket(bucket_name)
        try:
            # Attempt to create a new GCS bucket.
            bucket = self.storage_client.create_bucket(bucket_name)
            print(f'Bucket {bucket_name} created successfully.')
        except Exception as e:
            # Handle exceptions that may occur during bucket creation and print an error message.
            print(f'Error creating bucket: {str(e)}')
        
    def delete_gcs_bucket(self, bucket_name):
        """Deletes a bucket from Google Cloud Storage
    
        Args:
            bucket_name (str): Name of the bucket to delete
        """
        # Get the bucket
        bucket = self.storage_client.bucket(bucket_name)
    
        # Check if the bucket exists
        if not bucket.exists():
            return
        
        # Delete the bucket
        try:
            # deletes all the contents of a given bucket
            all_blobs = list(bucket.list_blobs())
            if len(all_blobs) >0:
                for blob in all_blobs:
                    blob.delete()
            
            bucket.delete()
            print(f"Bucket {bucket_name} deleted")
        except Exception as e:
            print(f"Failed to delete bucket {bucket_name}: {e}")

    def decompress_gz_files(self, target_directory, output_dir_name):
        """
        Decompresses all .gz files found in the specified target_directory.
    
        Parameters:
        target_directory (str): The path to the directory containing .gz files.
        """

        if len(output_dir_name) == 0:
            output_dir_name = bucket_name
        # output_dir_name = output_dir_name if output_dir_name.endswith('/') else output_dir_name + "/"  
        output_dir_path = os.path.join(self.storage_bucket_decompresssed_path, output_dir_name)
        Utils.create_directory(output_dir_path, override = True)        
        
        print("Start unziping the files under {} content to {}".format(target_directory, output_dir_path))

        # Iterate over all files in the target_directory
        for filename in tqdm(os.listdir(target_directory)):
            if filename.endswith('.gz'):
                # Form the full path to the .gz file
                file_path = os.path.join(target_directory, filename)
                # Create the destination file path by removing the .gz extension
                output_path = os.path.join(output_dir_path, filename[:-3])
                
                # Decompress the file
                with gzip.open(file_path, 'rb') as f_in, open(output_path, 'wb') as f_out:
                    shutil.copyfileobj(f_in, f_out)
            

In [2]:
# clinet = GoogleClient('sunlit-hub-431117-s8-52aeb76478ca.json', 'sunlit-hub-431117-s8', "storage_beacons")

In [3]:
# clinet.create_dataset()

In [4]:
# clinet.collect_and_download_beacons_contracts([ '0x77a34e599da1e37215445c5740d57b63e5bb98fd',
                # '0x60faeec73299cbe55d4e5736ebdf432a1d2117b3',
                # '0x28adcdc02ca7b3edf11924102726066aa0fa7010',
                # '0x563d38a5a31cdafec78bb4ed323a423620db42eb',
                # '0x90d424d95a223794c0508e53b2a3012eb5eacfa6',
                # '0x9378699605fd45139d49bec64cb5a70ec3350700',
                # '0x8a3462a50e1a9fe8c9e7d9023cacbd9a98d90021',
                # '0xbcd4c9ba1efb413b1ac952efaa2374f98641eb7f'], "df-beacon-contracts", override= True)